Câu 1: Hãy viết câu lệnh SQL để tính sự tương quan giữa A và B theo công thức sau


In [5]:
import sqlite3
import pandas as pd
import math

# Tạo DataFrame ban đầu
df = pd.DataFrame({
    'a': [5, 6, 4, 2, 5],
    'b': [2, 5, 6, 9, 1]
})

# Kết nối đến database SQLite
conn = sqlite3.connect("correlation_calc.db")
cursor = conn.cursor()

# Đăng ký hàm SQRT cho SQLite
conn.create_function('SQRT', 1, math.sqrt)

# Tạo bảng và chèn dữ liệu
cursor.execute('DROP TABLE IF EXISTS data_table')
cursor.execute('''
    CREATE TABLE data_table (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        a REAL,
        b REAL
    )
''')
df.to_sql('data_table', conn, if_exists='append', index=False)

# Truy vấn SQL tính hệ số tương quan Pearson
query = '''
SELECT 
    (COUNT(*) * SUM(a * b) - SUM(a) * SUM(b)) / 
    (SQRT(COUNT(*) * SUM(a * a) - POWER(SUM(a), 2)) * 
     SQRT(COUNT(*) * SUM(b * b) - POWER(SUM(b), 2))) AS correlation
FROM data_table
'''
cursor.execute(query)
correlation, = cursor.fetchone()

# In kết quả
print(f"Hệ số tương quan giữa a và b là (tính bằng SQL): {correlation:.4f}")

# Đóng kết nối
conn.close()


Hệ số tương quan giữa a và b là (tính bằng SQL): -0.7294


Câu 2: Một công ty oto đang kiểm tra 3 loại mẫu mới A, B và C trong 4 ngày, và chấm điểm theo thang từ 1
đến 10 điểm cho mỗi ngày với bảng sau. Liệu có sự khác biệt đáng kể giữa các mẫu dựa trên điểm số mà
chúng nhận được trong 4 ngày thử nghiệm không? Kết quả thử nghiệm phụ thuộc vào ngày hay phụ thuộc vào
mẫu xe? Hãy chuyển đổi dữ liệu sang dạng quan hệ và thực hiện kiểm tra χ2.

In [7]:
import sqlite3
import pandas as pd
from scipy.stats import chi2_contingency

# Bước 1: Dữ liệu gốc
data = {
    'Day': ['Day 1']*3 + ['Day 2']*3 + ['Day 3']*3 + ['Day 4']*3,
    'Model': ['A', 'B', 'C'] * 4,
    'Score': [8, 9, 7, 7.5, 8.5, 7, 6, 7, 8, 7, 6, 5]
}
df = pd.DataFrame(data)

# Bước 2: Phân loại mức điểm
def categorize(score):
    if score <= 6:
        return 'Low'
    elif score <= 8:
        return 'Medium'
    else:
        return 'High'

df['Level'] = df['Score'].apply(categorize)

# Bước 3: Lưu vào SQLite
conn = sqlite3.connect("car_score.db")
df.to_sql("car_scores", conn, if_exists='replace', index=False)

# Bước 4: Truy vấn lại từ SQLite
df_sql = pd.read_sql_query("SELECT * FROM car_scores", conn)

# Bước 5: Tạo bảng tần suất cho Chi-Square test
ct_model = pd.crosstab(df_sql['Model'], df_sql['Level'])
ct_day = pd.crosstab(df_sql['Day'], df_sql['Level'])

# Bước 6: Thực hiện kiểm định Chi-Square
chi2_model, p_model, _, _ = chi2_contingency(ct_model)
chi2_day, p_day, _, _ = chi2_contingency(ct_day)

# Bước 7: In kết quả
print("=== Kiểm định Chi-Square: Model vs Level ===")
print(ct_model)
print(f"Chi2 = {chi2_model:.4f}, p-value = {p_model:.4f}\n")

print("=== Kiểm định Chi-Square: Day vs Level ===")
print(ct_day)
print(f"Chi2 = {chi2_day:.4f}, p-value = {p_day:.4f}")

conn.close()


=== Kiểm định Chi-Square: Model vs Level ===
Level  High  Low  Medium
Model                   
A         0    1       3
B         2    1       1
C         0    1       3
Chi2 = 5.1429, p-value = 0.2730

=== Kiểm định Chi-Square: Day vs Level ===
Level  High  Low  Medium
Day                     
Day 1     1    0       2
Day 2     1    0       2
Day 3     0    1       2
Day 4     0    2       1
Chi2 = 6.0952, p-value = 0.4126


Câu 3: Bảng flights(departure_time,…) chứa các giá trị thời gian dưới dạng số nguyên (ví dụ: 830 cho 8:30
AM, 1445 cho 2:45 PM). Hãy chuyển đổi các giá trị này thành định dạng thời gian.

In [16]:
import sqlite3
import pandas as pd

# Kết nối đến database
conn = sqlite3.connect("flights.db")

# Truy vấn SQL chuyển đổi định dạng thời gian
query = """
SELECT 
    departure_time,
    printf('%02d:%02d', departure_time / 100, departure_time % 100) AS formatted_time
FROM flights
"""

# Thực hiện truy vấn và lấy kết quả bằng pandas
df = pd.read_sql_query(query, conn)
print(df)

# Đóng kết nối
conn.close()


   departure_time formatted_time
0             830          08:30
1            1015          10:15
2            1230          12:30
3            1445          14:45
4            1600          16:00
5            1730          17:30
6            2000          20:00


Câu 4: Viết truy vấn SQL để tìm các ngoại lệ bằng cách sử dụng MAD. Một quy tắc chung là xem xét các giá
trị ngoại lệ lớn hơn 1,5 lần so với giá trị MAD, trong đó x là số độ lệch chuẩn mà ta coi là có ý nghĩa.


In [17]:
import sqlite3
import pandas as pd

# Kết nối (tạo mới nếu chưa có)
conn = sqlite3.connect("flights.db")
cursor = conn.cursor()

# Tạo bảng flights với dữ liệu mẫu
cursor.execute("DROP TABLE IF EXISTS flights")
cursor.execute("""
    CREATE TABLE flights (
        flight_id INTEGER PRIMARY KEY,
        departure_time INTEGER,
        departure_delay INTEGER
    )
""")

# Dữ liệu mẫu
data = [
    (830, 5),
    (1015, -2),
    (1230, 10),
    (1445, 45),
    (1600, 60),
    (1730, -5),
    (2000, 90)
]

# Chèn dữ liệu
cursor.executemany("INSERT INTO flights (departure_time, departure_delay) VALUES (?, ?)", data)
conn.commit()
# Đọc lại bảng để tính MAD
df = pd.read_sql_query("SELECT departure_delay FROM flights", conn)

import numpy as np

median = np.median(df['departure_delay'])
mad = np.median(np.abs(df['departure_delay'] - median))

# Tìm outliers
df['outlier'] = np.abs(df['departure_delay'] - median) > 1.5 * mad
print(df)
print("\n Các điểm ngoại lệ:")
print(df[df['outlier']])


   departure_delay  outlier
0                5    False
1               -2    False
2               10    False
3               45     True
4               60     True
5               -5    False
6               90     True

 Các điểm ngoại lệ:
   departure_delay  outlier
3               45     True
4               60     True
6               90     True


Câu 5: Hãy xác định liệu hai người trong bảng Patient(last_name, weight, height) có phải là một người hay
không bằng cách sử dụng khoảng cách kết hợp Boolean trên “last_name” và “weight”.

In [13]:
import sqlite3
import pandas as pd

# Kết nối đến database
conn = sqlite3.connect("flights.db")
cursor = conn.cursor()

# Tạo bảng Patient và thêm dữ liệu mẫu nếu cần
cursor.execute("DROP TABLE IF EXISTS Patient")
cursor.execute("""
    CREATE TABLE Patient (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        last_name TEXT,
        weight REAL,
        height REAL
    )
""")

# Thêm vài bệnh nhân có cùng họ và cân nặng gần giống
sample_data = [
    ("Nguyen", 60, 165),
    ("Nguyen", 63, 167),
    ("Tran", 70, 175),
    ("Tran", 85, 180),
    ("Le", 50, 160),
    ("Le", 52, 158)
]
cursor.executemany("INSERT INTO Patient (last_name, weight, height) VALUES (?, ?, ?)", sample_data)
conn.commit()

# Truy vấn xác định những người có thể là trùng (cùng họ, cân nặng gần nhau)
query = """
SELECT 
    p1.last_name AS name1,
    p2.last_name AS name2,
    p1.weight AS weight1,
    p2.weight AS weight2,
    ABS(p1.weight - p2.weight) AS weight_diff
FROM Patient p1
JOIN Patient p2
  ON p1.rowid < p2.rowid
WHERE p1.last_name = p2.last_name
  AND ABS(p1.weight - p2.weight) <= 5;
"""

# Thực hiện truy vấn và hiển thị
df = pd.read_sql_query(query, conn)
print(df)

# Đóng kết nối
conn.close()



    name1   name2  weight1  weight2  weight_diff
0  Nguyen  Nguyen     60.0     63.0          3.0
1      Le      Le     50.0     52.0          2.0
